Homework 5: Neural Language Models  (& 🎃 SpOoKy 👻 authors 🧟 data) - Task 3
---

Task 3: Feedforward Neural Language Model (60 points)
--------------------------

For this task, you will create and train neural LMs for both your word-based embeddings and your character-based ones. You should write functions when appropriate to avoid excessive copy+pasting.

### a) First, encode  your text into integers (5 points)

In [1]:
# Importing utility functions from Keras
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

# necessary
from keras.models import Sequential
from keras.layers import Dense

# optional
# from keras.layers import Dropout

# if you want fancy progress bars
from tqdm import notebook
from IPython.display import display

# your other imports here
import time

import numpy as np
import neurallm_utils as nutils



[nltk_data] Downloading package punkt to /Users/maxnbf/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# load in necessary data
from gensim.models import KeyedVectors

# abstract into util functions
NGRAM = 3 # The ngram language model you want to train
EMBEDDING_SAVE_FILE_WORD = "spooky_embedding_word.txt" # The file to save your word embeddings to
EMBEDDING_SAVE_FILE_CHAR = "spooky_embedding_char.txt" # The file to save your word embeddings to
TRAIN_FILE = 'spooky_author_train.csv' # The file to train your language model on

data_by_char = nutils.read_file_spooky(TRAIN_FILE, NGRAM, by_character=True)
data_by_word = nutils.read_file_spooky(TRAIN_FILE, NGRAM, by_character=False)


In [3]:
# constants you may find helpful. Edit as you would like.
#EMBEDDINGS_SIZE = 50
EMBEDDINGS_SIZE = 50
NGRAM = 3 # The ngram language model you want to train

In [4]:
# Initialize a Tokenizer and fit on your data
# do this for both the word and character data

# It is used to vectorize a text corpus. Here, it just creates a mapping from 
# word to a unique index. (Note: Indexing starts from 0)
# Example:
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(data)
# encoded = tokenizer.texts_to_sequences(data)
char_tokenizer = Tokenizer()
char_tokenizer.fit_on_texts(data_by_char)
char_encoded = char_tokenizer.texts_to_sequences(data_by_char)

word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(data_by_word)
word_encoded = word_tokenizer.texts_to_sequences(data_by_word)


In [5]:
# print out the size of the word index for each of your tokenizers
# this should match what you calculated in Task 2 with your embeddings
print("Size of word index for character tokenizer: ", len(char_tokenizer.word_index))
print("Size of word index for word tokenizer: ", len(word_tokenizer.word_index))

Size of word index for character tokenizer:  60
Size of word index for word tokenizer:  25374


### b) Next, prepare the sequences to train your model from text (5 points)

#### Fixed n-gram based sequences

In [6]:
def generate_ngram_training_samples(encoded: list, ngram: int) -> list:
    '''
    Takes the encoded data (list of lists) and 
    generates the training samples out of it.
    Parameters:
    up to you, we've put in what we used
    but you can add/remove as needed
    return: 
    list of lists in the format [[x1, x2, ... , x(n-1), y], ...]
    '''

    samples = []
    for e in encoded: 
        samples.extend([e[i:i+ngram] for i in range(len(e) - ngram + 1)])
    
    return samples

char_samples = generate_ngram_training_samples(encoded=char_encoded, ngram=NGRAM)
word_samples = generate_ngram_training_samples(encoded=word_encoded, ngram=NGRAM)

print(len(char_samples))
print(len(word_samples))
# generate your training samples for both word and character data
# print out the first 5 training samples for each
# we have displayed the number of sequences
# to expect for both characters and words
#
# Spooky data by character should give 2957553 sequences
# [21, 21, 3]
# [21, 3, 9]
# [3, 9, 7]
# ...
# Spooky data by words shoud give 634080 sequences
# [1, 1, 32]
# [1, 32, 2956]
# [32, 2956, 3]
# ...

2957553
634080


### c) Then, split the sequences into X and y and create a Data Generator (20 points)

In [7]:
# 2.5 points

# Note here that the sequences were in the form: 
# sequence = [x1, x2, ... , x(n-1), y]
# We still need to separate it into [[x1, x2, ... , x(n-1)], ...], [y1, y2, ...]]
# do that here


def transform_seq_dimensions(samples: list): 
    X = []
    y = []

    for s in samples:
        X.append(s[:-1])
        y.append(s[-1])

    return X, y


X_char, y_char = transform_seq_dimensions(char_samples)
X_word, y_word = transform_seq_dimensions(word_samples)

# TODO: print out the shapes to verify that they are correct
print(np.array(X_word).shape)
print(np.array(y_word).shape)
print(np.array(X_char).shape)
print(np.array(y_char).shape)

(634080, 2)
(634080,)
(2957553, 2)
(2957553,)


In [8]:
# 2.5 points

# Initialize a function that reads the word embeddings you saved earlier
# and gives you back mappings from words to their embeddings and also 
# indexes from the tokenizers to their embeddings

def read_embeddings(filename: str, tokenizer: Tokenizer) -> (dict, dict):
    '''Loads and parses embeddings trained in earlier.
    Parameters:
        filename (str): path to file
        Tokenizer: tokenizer used to tokenize the data (needed to get the word to index mapping)
    Returns:
        (dict): mapping from word to its embedding vector
        (dict): mapping from index to its embedding vector
    '''
    # YOUR CODE HERE
    embedding = KeyedVectors.load_word2vec_format(filename, binary=False)

    word_to_vector = {}
    index_to_vector = {}
    for index, word in tokenizer.index_word.items():
        vector = embedding[word]
        word_to_vector[word] = vector
        index_to_vector[index] = vector

    # print(len(word_to_vector.keys()))
    # print(len(index_to_vector.keys()))

    return word_to_vector, index_to_vector

word_to_vector, word_index_to_vector = read_embeddings(EMBEDDING_SAVE_FILE_WORD, word_tokenizer)
char_to_vector, char_index_to_vector = read_embeddings(EMBEDDING_SAVE_FILE_CHAR, char_tokenizer)


In [9]:
# NECESSARY FOR CHARACTERS

# the "0" index of the Tokenizer is assigned for the padding token. Initialize
# the vector for padding token as all zeros of embedding size
# this adds one to the number of embeddings that were initially saved
# (and increases your vocab size by 1)

padding_token_vector = [0 for _ in range(EMBEDDINGS_SIZE)]

# char_to_vector[0] = padding_token_vector
char_index_to_vector[0] = padding_token_vector


# word_to_vector[0] = padding_token_vector
word_index_to_vector[0] = padding_token_vector


char_to_vector.keys()

dict_keys(['_', 'e', 't', 'a', 'o', 'n', 'i', 's', 'h', 'r', 'd', 'l', 'u', 'm', 'c', 'f', 'w', 'y', 'g', 'p', '<s>', '</s>', ',', 'b', 'v', '.', 'k', ';', '"', 'x', "'", 'q', 'j', 'z', '?', ':', 'é', 'æ', 'ê', 'ö', 'è', 'ë', 'à', 'ô', 'ñ', 'ä', 'ï', 'â', 'ü', 'ο', 'ἶ', 'δ', 'α', 'î', 'υ', 'π', 'ν', 'ς', 'å', 'ç'])

In [10]:
# 10 points

def data_generator(X: list, y: list, num_sequences_per_batch: int, index_2_embedding: dict) -> (list,list):
    '''
    Returns data generator to be used by feed_forward
    https://wiki.python.org/moin/Generators
    https://realpython.com/introduction-to-python-generators/
    
    Yields batches of embeddings and labels to go with them.
    Use one hot vectors to encode the labels 
    (see the to_categorical function)
    
    If for_feedforward is True: 
    Returns data generator to be used by feed_forward
    else: Returns data generator for RNN model
    '''
    # YOUR CODE HERE

    index = 0
    while True:

        # this is the data in the form [[21, 21], [21, 3], [3, 9], ...]
        embeddings = []
        start = index
        end = min(index+num_sequences_per_batch, len(X))

        for indeces in X[start:end]:
            row = []

            for i in indeces:  
                row.extend(index_2_embedding[i])

            embeddings.append(row)

        vocab_len = len(index_2_embedding.keys())
        labels = to_categorical(y[start:end], num_classes=vocab_len)
        
        # reshape maintains correct shape
        yield (np.array(embeddings).reshape(len(embeddings), EMBEDDINGS_SIZE*(NGRAM-1)), np.array(labels))
        index += num_sequences_per_batch
        index %= len(X)

    pass

In [11]:
# 5 points

# initialize your data_generator for both word and character data
# print out the shapes of the first batch to verify that it is correct for both word and character data

# Examples:
# num_sequences_per_batch = 128 # this is the batch size
# steps_per_epoch = len(sequences)//num_sequences_per_batch  # Number of batches per epoch
# train_generator = data_generator(X, y, num_sequences_per_batch)

# sample=next(train_generator) # this is how you get data out of generators
# sample[0].shape # (batch_size, (n-1)*EMBEDDING_SIZE)  (128, 200)
# sample[1].shape   # (batch_size, |V|) to_categorical

# character data
num_sequences_per_batch = 128 # this is the batch size
steps_per_epoch_word = len(X_word)//num_sequences_per_batch  # Number of batches per epoch
train_generator_word = data_generator(X_word, y_word, num_sequences_per_batch, word_index_to_vector)

print(steps_per_epoch_word)

sample=next(train_generator_word) # this is how you get data out of generators
print(sample[0].shape)
print(sample[1].shape)
#sample[0].shape # (batch_size, (n-1)*EMBEDDING_SIZE)  (128, 200)
#sample[1].shape   # (batch_size, |V|) to_categorical



steps_per_epoch_char= len(X_char)//num_sequences_per_batch  # Number of batches per epoch
train_generator_char = data_generator(X_char, y_char, num_sequences_per_batch, char_index_to_vector)

sample=next(train_generator_char) # this is how you get data out of generators
print(sample[0].shape)
print(sample[1].shape)

print(steps_per_epoch_char, steps_per_epoch_word)

4953
(128, 100)
(128, 25375)
(128, 100)
(128, 61)
23105 4953


### d) Train & __save__ your models (15 points)

In [12]:
# 15 points 

# code to train a feedforward neural language model for 
# both word embeddings and character embeddings
# make sure not to just copy + paste to train your two models
# (define functions as needed)

# train your models for between 3 & 5 epochs
# on Felix's machine, this takes ~ 24 min for character embeddings and ~ 10 min for word embeddings
# DO NOT EXPECT ACCURACIES OVER 0.5 (and even that is very for this many epochs)
# We recommend starting by training for 1 epoch

# Define your model architecture using Keras Sequential API
# Use the adam optimizer instead of sgd
# add cells as desired

def build_nn(input_size: int, vocab_size: int) -> Sequential:
    model = Sequential()

    # adds hidden layer
    # or input_shape=(input_size,)
    model.add(Dense(units=50, activation="elu", input_dim=input_size))

    model.add(Dense(units=50, activation="elu", input_dim=input_size))

    # adds an output layer
    #do i need units = vocab_size, does output and input need to match for multiple epochs?
    model.add(Dense(vocab_size, activation="softmax"))

    # shows the model's verbose
    model.summary()

    # calls compile here
    #https://stackoverflow.com/questions/45632549/why-is-the-accuracy-for-my-keras-model-always-0-when-training
    # different metric?
    model.compile(loss='categorical_crossentropy',
                    optimizer='adam',
                    metrics=["accuracy"])

    return model

word_model = build_nn((NGRAM-1) * EMBEDDINGS_SIZE, len(word_index_to_vector.keys()))
char_model = build_nn((NGRAM-1) * EMBEDDINGS_SIZE, len(char_index_to_vector.keys()))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                5050      
                                                                 
 dense_1 (Dense)             (None, 50)                2550      
                                                                 
 dense_2 (Dense)             (None, 25375)             1294125   
                                                                 
Total params: 1301725 (4.97 MB)
Trainable params: 1301725 (4.97 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 50)                5050      
                                                              

In [13]:
len(word_index_to_vector.keys())

25375

In [14]:
# Here is some example code to train a model with a data generator
# model.fit(x=train_generator, 
#           steps_per_epoch=steps_per_epoch,
#           epochs=1)


word_model.fit(x=train_generator_word, 
          steps_per_epoch=steps_per_epoch_word,
          epochs=5)



Epoch 1/5
4953/4953 [==============================] - 50s 10ms/step - loss: 5.7288 - accuracy: 0.1887
Epoch 2/5
4953/4953 [==============================] - 54s 11ms/step - loss: 5.3434 - accuracy: 0.1978
Epoch 3/5
4953/4953 [==============================] - 61s 12ms/step - loss: 5.2287 - accuracy: 0.1997
Epoch 4/5
4953/4953 [==============================] - 57s 12ms/step - loss: 5.1565 - accuracy: 0.2005
Epoch 5/5
4953/4953 [==============================] - 51s 10ms/step - loss: 5.1052 - accuracy: 0.2012


In [15]:

char_model.fit(x=train_generator_char, 
          steps_per_epoch=steps_per_epoch_char,
          epochs=5)


Epoch 1/5


23105/23105 [==============================] - 28s 1ms/step - loss: 2.0533 - accuracy: 0.3773
Epoch 2/5
23105/23105 [==============================] - 27s 1ms/step - loss: 1.9863 - accuracy: 0.3872
Epoch 3/5
23105/23105 [==============================] - 27s 1ms/step - loss: 1.9750 - accuracy: 0.3882
Epoch 4/5
23105/23105 [==============================] - 28s 1ms/step - loss: 1.9695 - accuracy: 0.3887
Epoch 5/5
23105/23105 [==============================] - 27s 1ms/step - loss: 1.9659 - accuracy: 0.3891


In [16]:

# spooky data model by character for 5 epochs takes ~ 24 min on Felix's computer
# with adam optimizer, gets accuracy of 0.3920

# spooky data model by word for 5 epochs takes 10 min on Felix's computer
# results in accuracy of 0.2110

In [17]:
# save your trained models so you can re-load instead of re-training each time
# also, you'll need these to generate your sentences!
char_model.save("char_model.h5")
word_model.save("word_model.h5")

/Users/maxnbf/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### e) Generate Sentences (15 points)

In [18]:
# load your models if you need to

word_model = keras.models.load_model("word_model.h5")
char_model = keras.models.load_model("char_model.h5")



In [19]:
# 10 points

# # generate a sequence from the model until you get an end of sentence token
# This is an example function header you might use
def generate_seq(model: Sequential, tokenizer: Tokenizer, seed: list):
    '''
    Parameters:
        model: your neural network
        tokenizer: the keras preprocessing tokenizer
        seed: [w1, w2, w(n-1)]
    Returns: string sentence
    '''
    sentence = seed.copy()

    while True:
        encoded = tokenizer.texts_to_sequences([" ".join(sentence)])[0]

        # to match input dimension of the NN
        padded = pad_sequences([encoded], maxlen=(NGRAM-1)*EMBEDDINGS_SIZE, padding='pre')
        prediction = model.predict(padded, verbose=0)
        index = np.argmax(prediction)

        next_word = tokenizer.index_word[index]
        sentence.append(next_word)

        print(" ".join(sentence))
    
        if next_word == "</s>":
            return " ".join(sentence)



In [20]:
# 5 points

# generate and display one sequence from both the word model and the character model
# do not include <s> or </s> in your displayed sentences
# make sure that you can read the output easily (i.e. don't just print out a list of tokens)

# you may leave _ as _ or replace it with a space if you prefer
#print(generate_seq(word_model, word_tokenizer, ["<s>", "<s>"]))
#print(generate_seq(word_model, word_tokenizer, ["<s>", "<s>", "I", "like", "eating", "food", "that", "tastes"]))
#print(generate_seq(word_model, word_tokenizer, ["<s>", "<s>", "Go", "away"]))
print(generate_seq(word_model, word_tokenizer, ["<s>", "<s>", "Go", "now", "to", "the"]))
print(generate_seq(char_model, char_tokenizer, ["<s>", "<s>", "I", "_", "l", "i", "k", "e"]))



<s> <s> will </s>
<s> <s> I like eating food that tastes tript first </s>
<s> <s> Go away oldeb first </s>


KeyboardInterrupt: 

In [ ]:
# generate 100 example sentences with each model and save them to a file, one sentence per line
# do not include <s> and </s> in your saved sentences (you'll use these sentences in your next task)
# this will produce two files, one for each model